In [ ]:
## Set libraries and read in data

#includes xx libraries: specify library
#change file locations 

#pip install if needed
import os
import numpy as np
import pandas as pd
import scanpy as sc
from scipy import stats
import anndata as ad
from anndata import AnnData
import configome as co
import harmonypy as hp
import seaborn as sns
import matplotlib.pyplot as plt
import scdrs
import besca as bc
import scanpy.external as sce

adata = sc.read_h5ad('/mnt/ps/home/CORP/storage/tenx/runa/scanpy/h5ad_files/processed/celltype.h5ad') 
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')
results_file = '/mnt/ps/home/CORP/storage/tenx/runa/scanpy/h5ad_files/processed/celltype.h5ad'
results_file_raw = '/mnt/ps/home/CORP/storage/tenx/runa/scanpy/h5ad_files/raw/celltype.h5ad'
## gs://comp-neuro/ #CompNeuro Google Cloud bucket

# RXRX samples
mysamples = "/mnt/bh1/data/storage/outs/"
myclusters = "/mnt/ps/home/CORP/storage/tenx/runa/scanpy/celltype"
os.chdir(mysamples)

adata = sc.read_10x_h5('filtered_feature_bc_matrix.h5', gex_only = True) 
adata.var_names_make_unique()
adata.write(results_file, compression='gzip')
adata.write(results_file_raw, compression='gzip')
os.chdir(myclusters)

## Add scDRS, disease relevance score ##
DATA_PATH = scdrs.__path__[0]
H5AD_FILE = os.path.join(DATA_PATH, '/mnt/ps/home/CORP/storage/tenx/runa/scanpy/h5ad_files/processed/celltype.h5ad')
GS_FILE = os.path.join(DATA_PATH, "/mnt/ps/home/CORP/amanda.mitchell/data/data/external/genetic-datasets/GWAS/magma_10kb_1000.74_traits.gs")

#Location of comp-neuro Google bucket gs://comp-neuro/

#Marker genes
ttestm = "celltype_markersttest.csv"
wilcm = "celltype_markerswilcoxon.csv"
ctwilcm = "celltype_markersttest.csv"


In [ ]:

adata.write(results_file_raw, compression='gzip')
adata.write(results_file, compression='gzip')
os.chdir(myclusters)

In [ ]:
## Preprocess and cluster

sc.pl.highest_expr_genes(adata, n_top=30, save='.png')
sc.pp.filter_cells(adata, min_genes=200)
#sc.pp.filter_genes(adata, min_cells=3)
adata.var['mt'] = adata.var_names.str.startswith('MT-') 

sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True, save='QCs.png')
adata = adata[adata.obs.n_genes_by_counts > 1000, :]
adata = adata[adata.obs.pct_counts_mt < 15, :]
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True, save='QCs_post.png')

## Normalize, log, variable genes ##
sc.pp.normalize_total(adata, target_sum=1e6) #CPM 1e6, by cells 1e4, len(adata), exclude_highly_expressed=True
sc.pp.log1p(adata)
#sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5) #how to change
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, batch_key = 'batch')
sc.pl.highly_variable_genes(adata, save='.png')
adata.raw = adata

## Filter ##
#adata.2 = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt']) #may overcorrect
sc.pp.scale(adata)#, max_value=10)

# PCA
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca(adata, save='.png')
adata.write(results_file)
adata

## Cluster ##
sc.pl.pca_variance_ratio(adata, log=True, n_pcs = 50, save='elbow.png')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=30) #change to minimal PCs based on Elbow
sc.tl.leiden(adata, resolution=1, n_iterations=100)

## Batch correction
adatab=adata
sce.pp.harmony_integrate(adatab, 'batch')
sc.pl.embedding(adatab, basis='X_pca_harmony', color=['batch'], save='batch_correction.png')
adatab.obsm['X_pca'] = adatab.obsm['X_pca_harmony']
sc.pp.neighbors(adatab, n_neighbors=10, n_pcs=30) #change to minimal PCs based on Elbow
sc.tl.leiden(adatab, resolution=1, n_iterations=100)

#UMAP
sc.tl.paga(adata)
sc.pl.paga(adata, plot=False)  
sc.tl.umap(adata, init_pos='paga')
sc.pl.umap(adata, color=['leiden'], save='leiden.png')
sc.pl.umap(adata, color=['batch'], save='leiden-batch.png')

sc.tl.paga(adatab)
sc.pl.paga(adatab, plot=False)  
sc.tl.umap(adatab, init_pos='paga')
sc.pl.umap(adatab, color=['leiden'], save='corrected-leiden.png')
sc.pl.umap(adatab, color=['batch'], save='corrected-leiden-batch.png')

broad_class = ["SLC17A6", "GAD2", "GFAP", "SOX2", "POU5F1", "NKX2-1", "NEUROG2", "TH"]
THsub = ['TH', "NR4A2", 'SLC6A3', 'GAD2']
mNPC = ["LMX1B", "FOXA2", "NR4A2", "NES"]
exc = ["SLC17A6", "SLC17A7", "SLC7A6", "SLC7A7"]
inh = ["GAD1", "GAD2", "SLC6A1", "GPHN"]

sc.pl.umap(adata, color=broad_class, use_raw=True, save='broad_class.png')
sc.pl.umap(adata, color=THsub, use_raw=True, save='THsub.png')
sc.pl.umap(adata, color=mNPC, use_raw=True, save='midbrainNPC_ct.png')
sc.pl.umap(adata, color=exc, use_raw=True, save='excitatory_umap.png')
sc.pl.umap(adata, color=inh, use_raw=True, save='inhibitory_umap.png')

sc.pl.dotplot(adata, broad_class, groupby='leiden', use_raw=True, save='broad_class.png')
sc.pl.dotplot(adata, THsub, groupby='leiden', use_raw=True, save='THsub.png')
sc.pl.dotplot(adata, iMN, groupby='leiden', use_raw=True, save='iMN.png')
sc.pl.dotplot(adata, mNPC, groupby='leiden', use_raw=True, save='mNPC.png')
sc.pl.dotplot(adata, exc, groupby='leiden', use_raw=True, save='exc.png')
sc.pl.dotplot(adata, inh, groupby='leiden', use_raw=True, save='inh.png')

adata.write(results_file)
adata.write(results_file_corrected)


In [ ]:
## QCs on UMAP

sc.pl.umap(adata, color=['n_genes_by_counts'], save='genecounts.png')
sc.pl.umap(adata, color=['pct_counts_mt'], save='pct_counts_mt.png')


In [ ]:
## Graph key genes and determine marker genes per cluster

#paper_iPSC = ["CNMD", "SFRP2", "PDLIM2", "CTSB"]
#paper_pluri = ["POU5F1", "SOX2", "SOX4", "PAX6", "FUT4", "NANOG"]

neuronal = ["TUBB3","MAP2","DLG4", "SYT1", "SYT7", "SYN1", "BSN", 'STXBP1']
subtypes = ['SLC6A4', 'CHAT', 'DRD1', 'DRD2', 'PPP1R1B']
iMN = ["ISL1", "MNX1", "FOXP1", "OLIG2", "COL3A1", "TAGLN", "SOX2", "TOP2A", "STMN2", "ONECUT2", "PHOX2A"]
iDA = ["TH", "TPH1", "SYT4", "SCN3A", "FOXA2", "LMX1A", "LMX1B", "NR4A2"]
uglia_a = ["TMEM119", "HEXB", "TREM2", "AIF1", "ARHGAP5", "CCR5", "CD164", "CSMD3", "CST3", "CX3CR1", "GCNT1", "GOLM1"]
uglia_b = ["GPR155", "GPR34", "ADGRG1", "GTF2H2", "LRBA", "LRRC3", "MED12L", "MFAP3", 
          "P2RY12", "P2RY13", "PLXDC2", "PMEPA1", "RAB39A", "SPARC", "TLR3", "AQP4"]
glu = ["SLC17A6", "SLC17A7", "KMT5C", "PTEN", "CHD2", "TCF4", "RELN", "SCN1A", "GRIN2B", "FOXP1", "STXBP1"]

sc.pl.umap(adata, color=neuronal, use_raw=True, save='neuronal_ct.png')
sc.pl.umap(adata, color=subtypes, use_raw=True, save='neuronalsubtypes_ct.png')
sc.pl.umap(adata, color=iMN, use_raw=True, save='iMN_ct.png')
sc.pl.umap(adata, color=iDA, use_raw=True, save='iDA_ct.png')
sc.pl.umap(adata, color=uglia_a, use_raw=True, save='iuglia_a_ct.png')
sc.pl.umap(adata, color=uglia_b, use_raw=True, save='iuglia_b_ct.png')
sc.pl.umap(adata, color=glu, use_raw=True, save='glu_ct.png')

# Marker genes, test
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False, save='ttest.png')
sc.settings.verbosity = 2  # reduce the verbosity
sc.pl.rank_genes_groups_matrixplot(adata, n_genes=3, standard_scale='var', cmap='Blues', save='ttest_mg_matrixplot.png')
sc.pl.rank_genes_groups_dotplot(adata, n_genes=3, standard_scale='var',save='ttest_mg_dotplot.png')
DEttest=pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(10)
DEttest.columns ='leiden_' + DEttest.columns
DEttest.to_csv(ttestm)

# Marker genes, wilcoxon
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups_dotplot(adata, n_genes=4, save='mg_dotplot.png')
sc.pl.rank_genes_groups_matrixplot(adata, n_genes=3, standard_scale='var', cmap='Blues', save='wilcoxon_mg_matrixplot.png')
sc.pl.rank_genes_groups_dotplot(adata, n_genes=3, standard_scale='var', cmap='Blues', save='wilcoxon_mg_dotplot.png')
DEwil=pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(10)
DEwil.columns ='leiden_' + DEwil.columns
DEwil.to_csv(wilcm)

## Dotplots ##
#sc.pl.stacked_violin(adata, neuronal, groupby='leiden', rotation=90, save='ranked_neuronal.png')
sc.pl.dotplot(adata, neuronal, groupby='leiden', save='neuronal.png')
sc.pl.dotplot(adata, subtypes, groupby='leiden', save='neuronalsubtypes.png')
sc.pl.dotplot(adata, iMN, groupby='leiden', save='iMN.png')
sc.pl.dotplot(adata, iDA, groupby='leiden', save='iDA.png')
sc.pl.dotplot(adata, uglia_a, groupby='leiden', save='uglia_a.png')
sc.pl.dotplot(adata, uglia_b, groupby='leiden', save='uglia_b.png')
sc.pl.dotplot(adata, glu, groupby='leiden', save='glu.png')

adata.write(results_file, compression='gzip')
#adata.raw.to_adata().write('./write/BatchE.h5ad')
#adata.uns['rank_genes_groups']

adata.uns['rank_genes_groups'].to_csv(./clustermarkers.csv)


In [ ]:
# Single cell disease risk score, scRDS

# Load .h5ad file, .cov file, and .gs file
#adata = scdrs.util.load_h5ad(H5AD_FILE, flag_filter_data=False, flag_raw_count=False)
df_gs = scdrs.util.load_gs(GS_FILE)

#Preproecssing .h5ad data compute scDRS score
scdrs.preprocess(adata) #preprocess

gene_list_SWB = df_gs['PASS_SWB'][0]
gene_weight_SWB = df_gs['PASS_SWB'][1]
df_res_SWB = scdrs.score_cell(adata, gene_list_SWB, gene_weight=gene_weight_SWB, n_ctrl=20)

gene_list_MDD = df_gs['PASS_MDD_Howard2019'][0]
gene_weight_MDD = df_gs['PASS_MDD_Howard2019'][1]
df_res_MDD = scdrs.score_cell(adata, gene_list_MDD, gene_weight=gene_weight_MDD, n_ctrl=20)

gene_list_AD = df_gs['PASS_Alzheimers_Jansen2019'][0]
gene_weight_AD = df_gs['PASS_Alzheimers_Jansen2019'][1]
df_res_AD = scdrs.score_cell(adata, gene_list_AD, gene_weight=gene_weight_AD, n_ctrl=20)

gene_list_Worry = df_gs['PASS_Worry_Nagel2018'][0]
gene_weight_Worry = df_gs['PASS_Worry_Nagel2018'][1]
df_res_Worry = scdrs.score_cell(adata, gene_list_Worry, gene_weight=gene_weight_Worry, n_ctrl=20)

gene_list_EDU_COLLEGE = df_gs['UKB_460K.cov_EDU_COLLEGE'][0]
gene_weight_EDU_COLLEGE = df_gs['UKB_460K.cov_EDU_COLLEGE'][1]
df_res_EDU_COLLEGE = scdrs.score_cell(adata, gene_list_EDU_COLLEGE, gene_weight=gene_weight_EDU_COLLEGE, n_ctrl=20)

gene_list_EDU_YEARS = df_gs['UKB_460K.cov_EDU_YEARS'][0]
gene_weight_EDU_YEARS = df_gs['UKB_460K.cov_EDU_YEARS'][1]
df_res_EDU_YEARS = scdrs.score_cell(adata, gene_list_EDU_YEARS, gene_weight=gene_weight_EDU_YEARS, n_ctrl=20)

gene_list_Verbal = df_gs['PASS_VerbalNumericReasoning_Davies2018'][0]
gene_weight_Verbal = df_gs['PASS_VerbalNumericReasoning_Davies2018'][1]
df_res_Verbal = scdrs.score_cell(adata, gene_list_Verbal, gene_weight=gene_weight_Verbal, n_ctrl=20)

gene_list_Intelligence = df_gs['PASS_Intelligence_SavageJansen2018'][0]
gene_weight_Intelligence = df_gs['PASS_Intelligence_SavageJansen2018'][1]
df_res_Intelligence = scdrs.score_cell(adata, gene_list_Intelligence, gene_weight=gene_weight_Intelligence, n_ctrl=20)

gene_list_risk = df_gs['PASS_GeneralRiskTolerance_KarlssonLinner2019'][0]
gene_weight_risk = df_gs['PASS_GeneralRiskTolerance_KarlssonLinner2019'][1]
df_res_risk = scdrs.score_cell(adata, gene_list_risk, gene_weight=gene_weight_risk, n_ctrl=20)

gene_list_SCZ = df_gs['PASS_Schizophrenia_Pardinas2018'][0]
gene_weight_SCZ = df_gs['PASS_Schizophrenia_Pardinas2018'][1]
df_res_SCZ = scdrs.score_cell(adata, gene_list_SCZ, gene_weight=gene_weight_SCZ, n_ctrl=20)

gene_list_BIP = df_gs['PASS_BIP_Stahl2019'][0]
gene_weight_BIP = df_gs['PASS_BIP_Stahl2019'][1]
df_res_BIP = scdrs.score_cell(adata, gene_list_BIP, gene_weight=gene_weight_BIP, n_ctrl=20)

gene_list_ADHD = df_gs['PASS_ADHD_Demontis2018'][0]
gene_weight_ADHD = df_gs['PASS_ADHD_Demontis2018'][1]
df_res_ADHD = scdrs.score_cell(adata, gene_list_ADHD, gene_weight=gene_weight_ADHD, n_ctrl=20)

adata.obs["norm_score_EDU_COLLEGE"] = df_res_EDU_COLLEGE["norm_score"]
adata.obs["norm_score_EDU_YEARS"] = df_res_EDU_YEARS["norm_score"]
adata.obs["norm_score_Verbal"] = df_res_Verbal["norm_score"]
adata.obs["norm_score_Intelligence"] = df_res_Intelligence["norm_score"]
adata.obs["norm_score_risk"] = df_res_risk["norm_score"]
adata.obs["norm_score_SCZ"] = df_res_SCZ["norm_score"]
adata.obs["norm_score_BIP"] = df_res_BIP["norm_score"]
adata.obs["norm_score_ADHD"] = df_res_ADHD["norm_score"]
adata.obs["norm_score_SWB"] = df_res_SWB["norm_score"]
adata.obs["norm_score_MDD"] = df_res_MDD["norm_score"]
adata.obs["norm_score_AD"] = df_res_AD["norm_score"]
adata.obs["norm_score_Worry"] = df_res_Worry["norm_score"]

sc.set_figure_params(figsize=[2.5, 2.5], dpi=150)
scDRS=["norm_score_EDU_COLLEGE", "norm_score_EDU_YEARS", "norm_score_Verbal", "norm_score_Intelligence",
"norm_score_risk", "norm_score_SCZ", "norm_score_BIP", "norm_score_ADHD",
"norm_score_SWB", "norm_score_MDD", "norm_score_AD","norm_score_Worry"]

sc.pl.umap(adata, color=scDRS, color_map="RdBu_r", vmin=-5, vmax=5, s=20, save='all_scDRS.png')
sc.pl.dotplot(adata, scDRS, groupby='leiden', use_raw=True, save='scDRS.png')
sc.pl.dotplot(adata, scDRS, groupby='CellType', use_raw=True, save='celltypes-scDRS.png')


#print(df_res.iloc[:4])
#df_gs.to_csv("data/geneset.gs", sep="\t")
## List all keys
#def getList(dict):
#    return dict.keys()
#print(getList(df_gs))
#for trait in dict_score:
#    adata.obs[trait] = dict_score[trait]["norm_score"]

adata.write(results_file, compression='gzip')


In [ ]:
## Trajectory mapping

sc.pl.paga(adata, color=['leiden'], save='tj.png')
adata.uns['iroot'] = np.flatnonzero(adata.obs['leiden']  == '0')[0]
sc.tl.dpt(adata)
#sc.pl.draw_graph(adata, color=['leiden', 'dpt_pseudotime'], legend_loc='on data', save='pseudotime.png')
sc.pl.umap(adata, color=['dpt_pseudotime'], save='pseudotime.png')

In [ ]:
## Percent each cluster

data = bc.tl.count_occurrence(adata, add_percentage=True, count_variable='leiden')
display(data)
fig = plt.figure(figsize=(15,6))
#visualize distribution before logarithmization
ax1 = fig.add_subplot(1, 2, 1)
ax1 = sns.barplot(data = data, x = data.index.tolist(), y = 'Counts' )
plt.xticks(rotation=90)
#visualize distribution after logarithmization
ax2 = fig.add_subplot(1, 2, 2)
ax2 = sns.barplot(data = data, x = data.index.tolist(), y = 'Percentage' )
plt.xticks(rotation=90)
plt.savefig('percent_cluster.png', format='png')


In [ ]:

## Reassign CellType
adata.obs['CellType'] = (
    adata.obs["leiden"]
    .map(lambda x: {
    '11': 'Excitatory 1', '13': 'Excitatory 1', '9': 'Excitatory 2',
    '3': 'Neural progenitor 1', '5': 'Neural progenitor 1', '6': 'Neural progenitor 1', '16': 'Neural progenitor 1', 
    '15': 'Neural progenitor 2',
    '0': 'Pluripotent 1', '12': 'Pluripotent 1', 
    '20': 'Pluripotent 2',
    '18': 'Pluripotent 3', '19': 'Pluripotent 3', '21': 'Pluripotent 3',
    '2': 'Pluripotent 4', '7': 'Pluripotent 4', '8': 'Pluripotent 4', 
    '4': 'Pluripotent 5', '14': 'Pluripotent 5', 
    '1': 'Pluripotent 6', '10': 'Pluripotent 6', '17': 'Pluripotent 6', 
    }.get(x, x)).astype("category"))

sc.pl.umap(adata, color=['CellType'], save='assigned_celltype.png')


## cM graphs
cmtx3 = sc.metrics.confusion_matrix("CellType", "leiden", adata.obs)
heatmap_plot4 = sns.heatmap(cmtx3)
fig4 = heatmap_plot4.get_figure()
fig4.savefig("celltype_cM.png") 

In [ ]:

## CellType Marker genes, wilcoxon
sc.tl.rank_genes_groups(adata, 'CellType', method='wilcoxon')
sc.pl.rank_genes_groups_dotplot(adata, n_genes=4, save='celltype_mg_dotplot.png')
sc.pl.rank_genes_groups_matrixplot(adata, n_genes=3, standard_scale='var', cmap='Blues', save='celltype_wilcoxon_mg_matrixplot.png')
sc.pl.rank_genes_groups_dotplot(adata, n_genes=3, standard_scale='var', cmap='Blues', save='celltype_wilcoxon_dotplot.png')
adata.write(results_file)
DEcelltype=pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(25)
DEcelltype.columns ='leiden_' + DEcelltype.columns
DEcelltype.to_csv(ctwilcm)

## Percent each cell type
data = bc.tl.count_occurrence(adata, add_percentage=True, count_variable='CellType')
display(data)
fig = plt.figure(figsize=(15,6))
#visualize distribution before logarithmization
ax1 = fig.add_subplot(1, 2, 1)
ax1 = sns.barplot(data = data, x = data.index.tolist(), y = 'Counts' )
plt.xticks(rotation=25)
#visualize distribution after logarithmization
ax2 = fig.add_subplot(1, 2, 2)
ax2 = sns.barplot(data = data, x = data.index.tolist(), y = 'Percentage' )
plt.xticks(rotation=25)
plt.savefig('percent_celltype.png', format='png')

sc.pl.dotplot(adata, scDRS, groupby='CellType', use_raw=True, save='celltypes-scDRS.png')
adata.write(results_file, compression='gzip')